In [13]:
import pandas as pd
import zipfile
import os
import re

In [9]:
#Descomprimimos
destino = 'datasets/'

if not os.path.exists(destino):
    os.makedirs(destino)

with zipfile.ZipFile("univaciada_1.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
    print(f"Archivos descomprimidos en {destino}")

with zipfile.ZipFile("univaciada_2.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
    print(f"Archivos descomprimidos en {destino}")

with zipfile.ZipFile("univaciada_3.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)
    print(f"Archivos descomprimidos en {destino}")

Archivos descomprimidos en datasets/
Archivos descomprimidos en datasets/
Archivos descomprimidos en datasets/


In [17]:
#Preprocesamiento
ficheros = os.listdir(destino)
ficheros

#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", "cod_tipo_estudio", "cod_municipio_residencia", "des_municipio_residencia", "lat_municipio_residencia", "lon_municipio_residencia"]

dataframes = []

#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles. Añadimos una columna con la universidad del estudiante.
for fichero in ficheros:
    try:
        df = pd.read_csv(destino+"/"+fichero, usecols=columnas_interes, low_memory=False)
        df = df[df["cod_tipo_estudio"] == 'G']  #Estudiantes de Grado
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO PERTENECIENTE A LA U.E.']
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO NO PERTENECIENTE A LA U.E.']
        df = df.dropna()
        nombre = fichero.split('/')[-1]
        universidad = re.match(r'^([^-]+)', nombre).group(1)
        df['universidad'] = universidad
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

#Concatenamos
df_matriculas = pd.concat(dataframes, ignore_index=True)

#Calculamos el tamaño de la burbuja normalizando los datos
df_matriculas['count'] = 1
df_grouped = df_matriculas.groupby(['lat_municipio_residencia', 'lon_municipio_residencia', 'universidad', 'curso_academico'], as_index=False).agg({
    'des_municipio_residencia': 'first',
    'count': 'sum'
})

min_count = df_grouped['count'].min()
max_count = df_grouped['count'].max()
df_grouped['scaled_count'] = 1 + 19 * (df_grouped['count'] - min_count) / (max_count - min_count)
df_grouped

,lat_municipio_residencia,lon_municipio_residencia,universidad,curso_academico,des_municipio_residencia,count,scaled_count
0,27.75379,-18.003640,uam,2017-18,FRONTERA,1,1.000000
1,27.75379,-18.003640,ucm,2017-18,FRONTERA,5,1.003009
2,27.75379,-18.003640,ucm,2018-19,FRONTERA,4,1.002257
3,27.75379,-18.003640,ucm,2019-20,FRONTERA,4,1.002257
4,27.75379,-18.003640,ucm,2020-21,FRONTERA,4,1.002257
...,...,...,...,...,...,...,...
45999,43.73261,-7.674875,uam,2022-23,"VICEDO, O",1,1.000000
46000,43.73261,-7.674875,uc3m,2020-21,"VICEDO, O",1,1.000000
46001,43.73261,-7.674875,uc3m,2021-22,"VICEDO, O",1,1.000000
46002,43.73261,-7.674875,uc3m,2022-23,"VICEDO, O",1,1.000000


In [18]:
df_grouped.to_csv('matricula.csv', index=False)  # index=False para no guardar el índice